In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import spacy
import nltk
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from collections import Counter



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/900k-spotify/final_milliondataset_BERT_500K_revised.json
/kaggle/input/900k-spotify/spotify_dataset.csv
/kaggle/input/900k-spotify/900k Definitive Spotify Dataset.json


In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [3]:
import string
import re

In [4]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

In [5]:
df = pd.read_csv('/kaggle/input/900k-spotify/spotify_dataset.csv')

In [6]:
df['emotion'].head(10)

0     sadness
1     sadness
2         joy
3         joy
4         joy
5        love
6     sadness
7         joy
8    surprise
9     sadness
Name: emotion, dtype: object

In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
df.head()

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Good for Morning Routine,Similar Artist 1,Similar Song 1,Similarity Score 1,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,sadness,hip hop,Thr!!!er,29th April 2013,D min,105,...,0,Corey Smith,If I Could Do It Again,0.986061,Toby Keith,Drinks After Work,0.983719,Space,Neighbourhood,0.983236
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,sadness,hip hop,Thr!!!er,29th April 2013,A# min,117,...,0,Hiroyuki Sawano,BRE@TH//LESS,0.995409,When In Rome,Heaven Knows,0.990905,Justice Crew,Everybody,0.984483
2,!!!,Pardon My Freedom,"Oh my god, did I just say that out loud? Shoul...",05:51,joy,hip hop,Louden Up Now,8th June 2004,A Maj,121,...,0,Ricky Dillard,More Abundantly Medley Live,0.993176,Juliet,Avalon,0.965147,The Jacksons,Lovely One,0.956752
3,!!!,Ooo,[Verse 1] Remember when I called you on the te...,03:44,joy,hip hop,As If,16th October 2015,A min,122,...,0,Eric Clapton,Man Overboard,0.992749,Roxette,Don't Believe In Accidents,0.991494,Tiwa Savage,My Darlin,0.990381
4,!!!,Freedom 15,[Verse 1] Calling me like I got something to s...,06:00,joy,hip hop,As If,16th October 2015,F min,123,...,0,Cibo Matto,Lint Of Love,0.981610,Barrington Levy,Better Than Gold,0.981524,Freestyle,Its Automatic,0.981415


In [9]:
df.columns

Index(['Artist(s)', 'song', 'text', 'Length', 'emotion', 'Genre', 'Album',
       'Release Date', 'Key', 'Tempo', 'Loudness (db)', 'Time signature',
       'Explicit', 'Popularity', 'Energy', 'Danceability', 'Positiveness',
       'Speechiness', 'Liveness', 'Acousticness', 'Instrumentalness',
       'Good for Party', 'Good for Work/Study',
       'Good for Relaxation/Meditation', 'Good for Exercise',
       'Good for Running', 'Good for Yoga/Stretching', 'Good for Driving',
       'Good for Social Gatherings', 'Good for Morning Routine',
       'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
       'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
       'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'],
      dtype='object')

In [10]:
df = df.drop(columns =['Good for Party', 'Good for Work/Study', 'Good for Relaxation/Meditation',
    'Good for Exercise', 'Good for Running', 'Good for Yoga/Stretching', 
    'Good for Driving', 'Good for Social Gatherings', 'Good for Morning Routine','Liveness', 'Instrumentalness', 'Time signature', 'Explicit','Artist(s)', 
    'song', 'Album', 'Release Date', 'Length', 'Key','Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
    'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2', 
    'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'
])

In [11]:
df.columns

Index(['text', 'emotion', 'Genre', 'Tempo', 'Loudness (db)', 'Popularity',
       'Energy', 'Danceability', 'Positiveness', 'Speechiness',
       'Acousticness'],
      dtype='object')

In [12]:
df['Genre'].nunique()

3097

In [13]:
df=df.dropna(subset=['Genre'])

In [14]:
def clean_genre(value):
    try:
        return [g.strip().lower() for g in str(value).split(',') if g.strip()]
    except:
        return []
df['Genre'] = df['Genre'].apply(clean_genre)

In [15]:
import ast

In [16]:
def unwrap_genres(raw):
    try:
        while isinstance(raw, str):
            raw = ast.literal_eval(raw)
        if isinstance(raw, list):
            return [g.strip().lower() for g in raw if isinstance(g, str)]
        return []
    except Exception:
        return []

df['Genre'] = df['Genre'].apply(unwrap_genres)

In [17]:
all_genres = df['Genre'].explode()
genre_counts = Counter(all_genres)
print(genre_counts)

Counter({'hip hop': 291844, 'rock': 84112, 'pop': 64388, 'alternative rock': 40271, 'indie rock': 22744, 'pop rock': 22592, 'electronic': 21474, 'folk': 20268, 'country': 16441, 'jazz': 16051, 'rap': 15969, 'soul': 15947, 'metal': 15583, 'trap': 15326, 'heavy metal': 13143, 'blues': 12786, 'hard rock': 11958, 'punk': 11427, 'indie pop': 10617, 'progressive rock': 10097, 'synthpop': 9215, 'hip-hop': 8794, 'reggae': 8511, 'electropop': 8375, 'funk': 6985, 'psychedelic rock': 6976, 'punk rock': 6658, 'indie': 6471, 'classic rock': 5822, 'new wave': 5782, 'classical': 5475, 'dance': 4667, 'metalcore': 4469, 'post-hardcore': 4384, 'progressive metal': 4117, 'emo': 4056, 'pop punk': 3876, 'alternative': 3870, 'post-punk': 3581, 'experimental': 3550, 'soundtrack': 3508, 'gospel': 3339, 'ambient': 3335, 'swing': 3320, 'rnb': 3099, 'thrash metal': 2896, 'drum and bass': 2639, 'death metal': 2508, 'cloud rap': 2489, 'house': 2485, 'trance': 2447, 'disco': 2358, 'lo-fi': 2281, 'dream pop': 2176, 

In [18]:
genre_counts = Counter(all_genres)
top_genres = {genre for genre, count in genre_counts.items() if count >= 100}
print(top_genres)

def filter_genres(genre_list):
    return [g if g in top_genres else 'other' for g in genre_list]

df['Filtered_Genre'] = df['Genre'].apply(filter_genres)

{'indie pop', 'doom metal', 'rnb', 'dream pop', 'emo', 'alt-country', 'math rock', 'acoustic', 'j-pop', 'trip-hop', 'dub', 'psychedelic', 'disco', 'grime', 'deathcore', 'chillout', 'progressive rock', 'classical', 'metalcore', 'latin', 'experimental', 'garage rock', 'cloud rap', 'death metal', 'black metal', 'electronic', 'swing', 'dance', 'alternative rock', 'punk rock', 'emo rap', 'indie rock', 'metal', 'dancehall', 'comedy', 'post-punk', 'house', 'industrial', 'classic rock', 'progressive metal', 'post-hardcore', 'melodic death metal', 'drum and bass', 'ambient', 'new wave', 'heavy metal', 'soul', 'gospel', 'chillwave', 'indie', 'electropop', 'country', 'rap', 'k-pop', 'funk', 'screamo', 'christian', 'thrash metal', 'pop', 'punk', 'dubstep', 'trap', 'pop punk', 'synthpop', 'soundtrack', 'power metal', 'hard rock', 'shoegaze', 'blues', 'trance', 'pop rock', 'jazz', 'hip-hop', 'grunge', 'alternative', 'rock', 'reggae', 'nu metal', 'psychedelic rock', 'hardcore', 'lo-fi', 'electro', 'h

In [19]:
df = df.drop(columns=['Genre'])

In [20]:
df.columns

Index(['text', 'emotion', 'Tempo', 'Loudness (db)', 'Popularity', 'Energy',
       'Danceability', 'Positiveness', 'Speechiness', 'Acousticness',
       'Filtered_Genre'],
      dtype='object')

In [21]:
df.isna().values.any()
df.isna().head()
nan_count = np.sum(df.isna(), axis = 0)
print(nan_count)

text              0
emotion           0
Tempo             0
Loudness (db)     0
Popularity        0
Energy            0
Danceability      0
Positiveness      0
Speechiness       0
Acousticness      0
Filtered_Genre    0
dtype: int64


In [22]:
all_emotions = df['emotion'].explode()
emotion_counts = Counter(all_emotions)
print(emotion_counts)

Counter({'joy': 209009, 'sadness': 171078, 'anger': 109678, 'fear': 28097, 'love': 27963, 'surprise': 5592, 'True': 17, 'Love': 3, 'pink': 2, 'angry': 1, 'thirst': 1, 'confusion': 1, 'interest': 1})


In [23]:
top_emotions = {emotion for emotion, count in emotion_counts.items() if count >= 100}
print(top_emotions)

def filter_emotion(emotion):
    return emotion if emotion in top_emotions else 'other'

df['Filtered_Emotions'] = df['emotion'].apply(filter_emotion)

{'joy', 'fear', 'anger', 'love', 'sadness', 'surprise'}


In [24]:
df['Filtered_Emotions'].value_counts()

Filtered_Emotions
joy         209009
sadness     171078
anger       109678
fear         28097
love         27963
surprise      5592
other           26
Name: count, dtype: int64

In [25]:
df['Filtered_Lyrics']= df['text'].str.lower().str.strip()
df = df.drop_duplicates(subset=['Filtered_Lyrics'])

The lyrics text has a division between chorus, bridge and such. So for the model to not get confused I'm removing those as well in the pre process method

In [26]:
def remove_headers(text):
    return re.sub(r'[\[\(]?(verse|chorus|bridge|intro|outro|hook|pre-chorus|refrain)[^\]\)]*[\]\)]?', '', text, flags=re.IGNORECASE)

Remove special characters,extra spaces, punctuation and stopwords then tokenize

In [27]:
def pre_processing (text):
     stop_words = set(stopwords.words('english'))
     text = remove_headers(text)
     text = re.sub(r'\d+', '', text)
     text = text.translate(str.maketrans('', '', string.punctuation))
     text = re.sub(r'\s+', ' ', text).strip()
     tokens= nltk.word_tokenize(text)
     tokens = [word for word in tokens if word not in stop_words]
     text = ' '.join(tokens)
     return text

In [28]:
df['Filtered_Lyrics'] = df['Filtered_Lyrics'].apply(pre_processing)
    

The loudness column had db after the number, meaning it is a string and we wouldn't be able to conduct numerical operations with it so I converted it

In [29]:
df['Loudness (db)'] = pd.to_numeric(df['Loudness (db)'].str.replace('db', '', regex=False), errors='coerce')

Checking if it is needed to convert any other column into numeric

In [30]:
print(df['Danceability'].dtype,
df['Energy'].dtype,
df['Popularity'].dtype,
df['Positiveness'].dtype,
df['Speechiness'].dtype,
df['Acousticness'].dtype)

int64 int64 int64 int64 int64 int64


In [31]:
df.shape

(497466, 13)

In [32]:
df.describe()

,Tempo,Loudness (db),Popularity,Energy,Danceability,Positiveness,Speechiness,Acousticness
count,497466.000000,497466.000000,497466.000000,497466.000000,497466.000000,497466.000000,497466.000000,497466.000000
mean,120.782359,-8.107713,30.481703,62.756261,58.291536,47.067373,11.402984,26.039154
std,29.262926,4.014701,17.185743,22.682011,17.352046,24.090915,12.285441,29.607546
min,31.000000,-50.060000,0.000000,0.000000,6.000000,0.000000,2.000000,0.000000
25%,97.000000,-9.950000,19.000000,48.000000,46.000000,28.000000,4.000000,2.000000
50%,120.000000,-7.250000,28.000000,65.000000,59.000000,46.000000,6.000000,13.000000
75%,140.000000,-5.400000,40.000000,81.000000,71.000000,66.000000,14.000000,44.000000
max,200.000000,4.980000,100.000000,100.000000,99.000000,100.000000,97.000000,100.000000


In [33]:
tokenized_texts = [text.split() for text in df['text']]
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1)